In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Page 2').getOrCreate()
spark

24/08/20 09:02:23 WARN Utils: Your hostname, codespaces-5b8cb8 resolves to a loopback address: 127.0.0.1; using 10.0.0.51 instead (on interface eth0)
24/08/20 09:02:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/20 09:02:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
training = spark.read.csv('test1.csv',header=True,inferSchema=True)
training.show()
training.printSchema()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [4]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [5]:
## [Age, Experience] --> new feature --> independent feature

In [6]:
from pyspark.ml.feature import VectorAssembler

In [15]:
assembler = VectorAssembler(
    inputCols=["age","Experience"],
    outputCol="Independent features")

output = assembler.transform(training)

In [16]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [17]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent features']

In [20]:
finalized_data = output.select(["Independent Features","Salary"])

In [21]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [23]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data, test_data = finalized_data.randomSplit([0.75,0.25]);
regressor = LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

24/08/20 09:09:30 WARN Instrumentation: [275bc02f] regParam is zero, which might cause numerical instability and overfitting.
24/08/20 09:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/20 09:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
24/08/20 09:09:30 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [24]:
###Coefficients
regressor.coefficients

DenseVector([-58.8235, 1666.6667])

In [25]:
###intercepts
regressor.intercept

15196.078431372378

In [26]:
##Prediction
pred_results = regressor.evaluate(test_data)

In [27]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [24.0,3.0]| 20000|18784.313725490192|
|          [30.0,8.0]| 25000|26764.705882352937|
+--------------------+------+------------------+

